## Задание 1. Определите распространенные правила с помощью алгоритмов Apriori и FPgrowth. Выделите не менее 5 логичных и 5 нелогичных правил.

In [ ]:
import pandas as pd
import numpy as np

from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df_original = pd.read_excel(filepath).rename({'heigh' : 'height'}, axis=1).drop('age', axis=1)
df = df_original.copy()
print('shape', df_original.shape)
df.head(-5)

shape (984, 16)


,target,educ,work_exp,interest,coffee,boss_educ,passport,green,floor,children,climate,offhour,dist,salary,height,lunch
0,0,4,27,8,1.3,1,0,2,3,2,0,0,10.6,1,176,226
1,0,3,10,0,0.0,6,1,2,3,0,0,0,11.5,0,186,226
2,0,3,22,7,1.9,6,1,1,5,0,3,0,12.8,1,166,238
3,0,3,31,7,2.0,4,0,3,3,1,1,0,15.2,1,180,216
4,1,3,23,7,0.0,6,1,3,2,0,0,0,16.1,1,180,286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,1,3,23,5,0.0,6,1,1,3,1,2,0,12.4,1,177,309
975,0,4,17,9,0.0,5,0,1,1,4,2,1,11.6,1,167,260
976,1,5,35,9,0.0,5,0,4,4,0,0,1,12.1,0,168,197
977,0,4,27,8,0.0,3,1,4,25,2,0,1,12.3,0,180,441


In [ ]:
!pip install python-docx
import docx

desc = docx.Document(docpath)
print(*[par.text for par in desc.paragraphs], sep = '\n')

target – сотрудник уволится в следующем году (0-нет, 1-да)
age - возраст
educ - образование (1-начальное, 2-среднее, 3 -неоконченное высшее, 4-два и более высших образования, 5-кандидат наук, 6-доктор наук)
work_exp – стаж работы (лет)
interest – оценка сотрудником интереса к работе (от 0 до 10)
coffee - среднее количество чашек кофе, выпиваемых сотрудником за рабочий день
boss_educ - образование руководителя (1-начальное, 2-среднее, 3 -неоконченное высшее, 4-два и более высших образования, 5-кандидат наук, 6-доктор наук)
passport - наличие загранпаспорта у сотрудника (1-есть, 0 -нет)
green - степень озеленения офиса (от 1 до 5)
floor – этаж работы
children – количество детей у сотрудника
climate - удовлетворенность рабочей атмосферой (от 0 до 4)
offhour – наличие переработок (1-да, 0 -нет)
dist – расстояние до работы
salary – удовлетворенность заработной платой (1-да, 0 -нет)
height - рост сотрудника
lunch - средняя цена бизнес-ланча в офисе


Подготовка датасета для использования ассоциативных правил:

* Бинарные переменные оставим как есть  
* Для переведения остальных переменных в бинарные сравним значения ряда с их медианой.

In [ ]:
for col in '''target
passport
offhour
salary'''.split('\n'):
  df[col] = df[col].astype('bool')

for col in '''educ
work_exp
interest
coffee
boss_educ
green
floor
children
climate
dist
height
lunch'''.split('\n'):
  df[col] = df[col] > df[col].median()


df.head(-5)

,target,educ,work_exp,interest,coffee,boss_educ,passport,green,floor,children,climate,offhour,dist,salary,height,lunch
0,False,True,True,True,True,False,False,False,False,True,False,False,False,True,False,False
1,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False
2,False,False,True,True,True,True,True,False,True,False,True,False,True,True,False,False
3,False,False,True,True,True,False,False,True,False,False,False,False,True,True,True,False
4,True,False,True,True,False,True,True,True,False,False,False,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,True,False,True,False,False,True,True,False,False,False,False,False,True,True,False,True
975,False,True,False,True,False,True,False,False,False,True,False,True,False,True,False,True
976,True,True,True,True,False,True,False,True,False,False,False,True,False,False,False,False
977,False,True,True,True,False,False,True,True,True,True,False,True,True,False,True,True


Сначала я инициализирую и выведу все таблицы, после чего я займусь поиском правил с помощью сделанных таблиц

### Использование Apriori

In [ ]:
apriori_freq_items = apriori(df, min_support = 0.1, use_colnames = True)
apriori_freq_items.head(-5)

,support,itemsets
0,0.320122,(target)
1,0.269309,(educ)
2,0.489837,(work_exp)
3,0.475610,(interest)
4,0.497967,(coffee)
...,...,...
199,0.106707,"(offhour, salary, dist)"
200,0.109756,"(offhour, height, dist)"
201,0.115854,"(offhour, lunch, dist)"
202,0.104675,"(offhour, height, salary)"


In [ ]:
#используем метрику support
apriori_ar_support = association_rules(apriori_freq_items, metric = 'support', min_threshold = 0.2)
apriori_ar_support.head(-5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(target),(offhour),0.320122,0.494919,0.220528,0.688889,1.391923,0.062094,1.623476,0.414147
1,(offhour),(target),0.494919,0.320122,0.220528,0.445585,1.391923,0.062094,1.226299,0.557474
2,(work_exp),(interest),0.489837,0.475610,0.228659,0.466805,0.981487,-0.004313,0.983487,-0.035654
3,(interest),(work_exp),0.475610,0.489837,0.228659,0.480769,0.981487,-0.004313,0.982535,-0.034720
4,(work_exp),(coffee),0.489837,0.497967,0.260163,0.531120,1.066576,0.016240,1.070707,0.122354
...,...,...,...,...,...,...,...,...,...,...
64,(height),(dist),0.478659,0.492886,0.231707,0.484076,0.982126,-0.004217,0.982924,-0.033731
65,(dist),(height),0.492886,0.478659,0.231707,0.470103,0.982126,-0.004217,0.983855,-0.034644
66,(lunch),(dist),0.497967,0.492886,0.247967,0.497959,1.010292,0.002526,1.010105,0.020293
67,(dist),(lunch),0.492886,0.497967,0.247967,0.503093,1.010292,0.002526,1.010314,0.020089


In [ ]:
#нет сетов с поддержкой 0.5 или более

In [ ]:
#используем метрику confidence
apriori_ar_confidence = association_rules(apriori_freq_items, metric = 'confidence', min_threshold = 0.6)
apriori_ar_confidence.head(-5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(target),(offhour),0.320122,0.494919,0.220528,0.688889,1.391923,0.062094,1.623476,0.414147
1,(target),(dist),0.320122,0.492886,0.192073,0.600000,1.217320,0.034289,1.267785,0.262581
2,(floor),(coffee),0.320122,0.497967,0.196138,0.612698,1.230398,0.036728,1.296232,0.275425
3,"(target, work_exp)",(offhour),0.159553,0.494919,0.113821,0.713376,1.441400,0.034855,1.762173,0.364366
4,"(target, work_exp)",(dist),0.159553,0.492886,0.101626,0.636943,1.292271,0.022985,1.396787,0.269105
5,"(target, passport)",(offhour),0.162602,0.494919,0.113821,0.700000,1.414374,0.033347,1.683604,0.349861
6,"(offhour, passport)",(target),0.189024,0.320122,0.113821,0.602151,1.881004,0.053310,1.708883,0.577538
7,"(target, offhour)",(dist),0.220528,0.492886,0.133130,0.603687,1.224799,0.024435,1.279578,0.235467
8,"(target, dist)",(offhour),0.192073,0.494919,0.133130,0.693122,1.400476,0.038069,1.645868,0.353939
9,"(target, height)",(offhour),0.156504,0.494919,0.104675,0.668831,1.351396,0.027218,1.525147,0.308270


In [ ]:
#используем метрику lift
apriori_ar_lift = association_rules(apriori_freq_items, metric = 'lift', min_threshold = 1.25)
apriori_ar_lift.head(-5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(target),(passport),0.320122,0.377033,0.162602,0.507937,1.347195,0.041905,1.266031,0.379064
1,(passport),(target),0.377033,0.320122,0.162602,0.431267,1.347195,0.041905,1.195425,0.413693
2,(target),(offhour),0.320122,0.494919,0.220528,0.688889,1.391923,0.062094,1.623476,0.414147
3,(offhour),(target),0.494919,0.320122,0.220528,0.445585,1.391923,0.062094,1.226299,0.557474
4,(boss_educ),(educ),0.326220,0.269309,0.151423,0.464174,1.723576,0.063569,1.363673,0.623068
5,(educ),(boss_educ),0.269309,0.326220,0.151423,0.562264,1.723576,0.063569,1.539240,0.574540
6,"(target, work_exp)",(offhour),0.159553,0.494919,0.113821,0.713376,1.441400,0.034855,1.762173,0.364366
7,"(offhour, work_exp)",(target),0.244919,0.320122,0.113821,0.464730,1.451729,0.035417,1.270160,0.412096
8,(target),"(offhour, work_exp)",0.320122,0.244919,0.113821,0.355556,1.451729,0.035417,1.171678,0.457679
9,(offhour),"(target, work_exp)",0.494919,0.159553,0.113821,0.229979,1.441400,0.034855,1.091461,0.606299


In [ ]:
#поставить максимальное значение lift в качестве отсечки нельзя, поэтому,
#чтобы получить негативные правила, возьмем случайные правила и отсортируем по lift
apriori_ar_lift_neg = association_rules(apriori_freq_items, metric = 'support', min_threshold = 0.1)
apriori_ar_lift_neg = apriori_ar_lift_neg[apriori_ar_lift_neg['lift'] < 1].sort_values('lift')
apriori_ar_lift_neg.head(-5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
99,(passport),(coffee),0.377033,0.497967,0.125000,0.331536,0.665779,-0.062750,0.751024,-0.446235
98,(coffee),(passport),0.497967,0.377033,0.125000,0.251020,0.665779,-0.062750,0.831755,-0.499984
86,(offhour),(interest),0.494919,0.475610,0.181911,0.367556,0.772811,-0.053478,0.829150,-0.367905
87,(interest),(offhour),0.475610,0.494919,0.181911,0.382479,0.772811,-0.053478,0.817917,-0.359224
27,(educ),(coffee),0.269309,0.497967,0.110772,0.411321,0.825999,-0.023335,0.852812,-0.223781
...,...,...,...,...,...,...,...,...,...,...
291,(work_exp),"(coffee, green)",0.489837,0.208333,0.101626,0.207469,0.995851,-0.000423,0.998909,-0.008101
290,"(coffee, green)",(work_exp),0.208333,0.489837,0.101626,0.487805,0.995851,-0.000423,0.996032,-0.005236
65,(dist),(work_exp),0.492886,0.489837,0.240854,0.488660,0.997596,-0.000580,0.997697,-0.004730
64,(work_exp),(dist),0.489837,0.492886,0.240854,0.491701,0.997596,-0.000580,0.997669,-0.004702


### Использование FPGrowth

In [ ]:
fpgrowth_freq_items = fpgrowth(df, min_support = 0.1, use_colnames = True)
fpgrowth_freq_items.head(-5)

,support,itemsets
0,0.497967,(coffee)
1,0.489837,(work_exp)
2,0.475610,(interest)
3,0.440041,(salary)
4,0.415650,(children)
...,...,...
199,0.115854,"(target, children)"
200,0.133130,"(target, offhour, dist)"
201,0.113821,"(target, offhour, passport)"
202,0.101626,"(target, work_exp, dist)"


In [ ]:
#используем метрику support
fpgrowth_ar_support = association_rules(fpgrowth_freq_items, metric = 'support', min_threshold = 0.2)
fpgrowth_ar_support.head(-5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(coffee),(lunch),0.497967,0.497967,0.234756,0.471429,0.946706,-0.013216,0.949791,-0.100827
1,(lunch),(coffee),0.497967,0.497967,0.234756,0.471429,0.946706,-0.013216,0.949791,-0.100827
2,(work_exp),(coffee),0.489837,0.497967,0.260163,0.531120,1.066576,0.016240,1.070707,0.122354
3,(coffee),(work_exp),0.497967,0.489837,0.260163,0.522449,1.066576,0.016240,1.068289,0.124336
4,(work_exp),(dist),0.489837,0.492886,0.240854,0.491701,0.997596,-0.000580,0.997669,-0.004702
...,...,...,...,...,...,...,...,...,...,...
64,(lunch),(green),0.497967,0.416667,0.200203,0.402041,0.964898,-0.007283,0.975540,-0.067567
65,(green),(lunch),0.416667,0.497967,0.200203,0.480488,0.964898,-0.007283,0.966354,-0.058703
66,(offhour),(green),0.494919,0.416667,0.222561,0.449692,1.079261,0.016345,1.060012,0.145402
67,(green),(offhour),0.416667,0.494919,0.222561,0.534146,1.079261,0.016345,1.084206,0.125897


In [ ]:
#используем метрику confidence
fpgrowth_ar_confidence = association_rules(fpgrowth_freq_items, metric = 'confidence', min_threshold = 0.6)
fpgrowth_ar_confidence.head(-5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(work_exp, interest)",(coffee),0.228659,0.497967,0.137195,0.600000,1.204898,0.023331,1.255081,0.220466
1,"(work_exp, children)",(coffee),0.214431,0.497967,0.135163,0.630332,1.265809,0.028383,1.358062,0.267311
2,"(interest, children)",(coffee),0.191057,0.497967,0.126016,0.659574,1.324533,0.030876,1.474721,0.302885
3,"(climate, coffee)",(interest),0.174797,0.475610,0.109756,0.627907,1.320215,0.026621,1.409299,0.293924
4,"(climate, interest)",(coffee),0.178862,0.497967,0.109756,0.613636,1.232282,0.020689,1.299378,0.229556
5,(floor),(coffee),0.320122,0.497967,0.196138,0.612698,1.230398,0.036728,1.296232,0.275425
6,(target),(dist),0.320122,0.492886,0.192073,0.600000,1.217320,0.034289,1.267785,0.262581
7,(target),(offhour),0.320122,0.494919,0.220528,0.688889,1.391923,0.062094,1.623476,0.414147
8,"(target, offhour)",(dist),0.220528,0.492886,0.133130,0.603687,1.224799,0.024435,1.279578,0.235467
9,"(target, dist)",(offhour),0.192073,0.494919,0.133130,0.693122,1.400476,0.038069,1.645868,0.353939


In [ ]:
#используем метрику lift
fpgrowth_ar_lift = association_rules(fpgrowth_freq_items, metric = 'lift', min_threshold = 1.25)
fpgrowth_ar_lift.head(-5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(work_exp, children)",(coffee),0.214431,0.497967,0.135163,0.630332,1.265809,0.028383,1.358062,0.267311
1,(coffee),"(work_exp, children)",0.497967,0.214431,0.135163,0.271429,1.265809,0.028383,1.078232,0.418283
2,"(interest, children)",(coffee),0.191057,0.497967,0.126016,0.659574,1.324533,0.030876,1.474721,0.302885
3,(coffee),"(interest, children)",0.497967,0.191057,0.126016,0.253061,1.324533,0.030876,1.083011,0.488050
4,(boss_educ),(educ),0.326220,0.269309,0.151423,0.464174,1.723576,0.063569,1.363673,0.623068
5,(educ),(boss_educ),0.269309,0.326220,0.151423,0.562264,1.723576,0.063569,1.539240,0.574540
6,"(climate, coffee)",(interest),0.174797,0.475610,0.109756,0.627907,1.320215,0.026621,1.409299,0.293924
7,"(coffee, interest)",(climate),0.267276,0.327236,0.109756,0.410646,1.254895,0.022294,1.141529,0.277213
8,(climate),"(coffee, interest)",0.327236,0.267276,0.109756,0.335404,1.254895,0.022294,1.102509,0.301919
9,(interest),"(climate, coffee)",0.475610,0.174797,0.109756,0.230769,1.320215,0.026621,1.072764,0.462532


In [ ]:
#используем тот же трюк, что и в apriori для получения негативных правил
fpgrowth_ar_lift_neg = association_rules(fpgrowth_freq_items, metric = 'support', min_threshold = 0.1)
fpgrowth_ar_lift_neg = fpgrowth_ar_lift_neg[fpgrowth_ar_lift_neg['lift'] < 1].sort_values('lift')
fpgrowth_ar_lift_neg.head(-5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
429,(passport),(coffee),0.377033,0.497967,0.125000,0.331536,0.665779,-0.062750,0.751024,-0.446235
428,(coffee),(passport),0.497967,0.377033,0.125000,0.251020,0.665779,-0.062750,0.831755,-0.499984
56,(offhour),(interest),0.494919,0.475610,0.181911,0.367556,0.772811,-0.053478,0.829150,-0.367905
57,(interest),(offhour),0.475610,0.494919,0.181911,0.382479,0.772811,-0.053478,0.817917,-0.359224
330,(coffee),(educ),0.497967,0.269309,0.110772,0.222449,0.825999,-0.023335,0.939734,-0.295578
...,...,...,...,...,...,...,...,...,...,...
583,(work_exp),"(coffee, green)",0.489837,0.208333,0.101626,0.207469,0.995851,-0.000423,0.998909,-0.008101
582,"(coffee, green)",(work_exp),0.208333,0.489837,0.101626,0.487805,0.995851,-0.000423,0.996032,-0.005236
5,(dist),(work_exp),0.492886,0.489837,0.240854,0.488660,0.997596,-0.000580,0.997697,-0.004730
4,(work_exp),(dist),0.489837,0.492886,0.240854,0.491701,0.997596,-0.000580,0.997669,-0.004702


### Нахождение правил

In [ ]:
#правила подбираются простым перебором уже сделанных таблиц, например:
fpgrowth_ar_lift_neg.iloc[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
429,(passport),(coffee),0.377033,0.497967,0.125000,0.331536,0.665779,-0.062750,0.751024,-0.446235
428,(coffee),(passport),0.497967,0.377033,0.125000,0.251020,0.665779,-0.062750,0.831755,-0.499984
56,(offhour),(interest),0.494919,0.475610,0.181911,0.367556,0.772811,-0.053478,0.829150,-0.367905
57,(interest),(offhour),0.475610,0.494919,0.181911,0.382479,0.772811,-0.053478,0.817917,-0.359224
330,(coffee),(educ),0.497967,0.269309,0.110772,0.222449,0.825999,-0.023335,0.939734,-0.295578
331,(educ),(coffee),0.269309,0.497967,0.110772,0.411321,0.825999,-0.023335,0.852812,-0.223781
519,(climate),(offhour),0.327236,0.494919,0.135163,0.413043,0.834568,-0.026793,0.860509,-0.227585
518,(offhour),(climate),0.494919,0.327236,0.135163,0.273101,0.834568,-0.026793,0.925526,-0.281847
658,(target),(salary),0.320122,0.440041,0.117886,0.368254,0.836864,-0.022980,0.886368,-0.222833
659,(salary),(target),0.440041,0.320122,0.117886,0.267898,0.836864,-0.022980,0.928666,-0.258231


Примечание: изначально датасет включал переменную age, но после получения правил стало понятно, что наличие переменных age и work_exp сильно снижает качество полученных правил, поэтому переменная age была удалена. Тем не менее, один очевидный вывод (Сотрудники с большим возрастом часто имеют большой рабочий стаж и наоборот), полученный в первоначальном датасете, был зафиксирован.

**Правила:**

Логичные:
* Сотрудники с большим возрастом часто имеют большой рабочий стаж и наоборот
* Сотрудники с большим рабочим стажем и большим количество детей пьют много кофе
* Сотрудники с высоким уровнем образования часто имеют босса с высоким уровнем образования
* Перерабатывающие сотрудники, которые живут далеко от работы, часто увольняются в следующем году
* Переработки уменьшают интерес сотрудника к работе
* Удовлетворенность зарплатой не влияет на шанс переработок
* Удовлетворенность зарплатой немного снижает шанс на уход сотрудника в следующем году


Нелогичные:
* Сотрудники с большим интересом к работе и большим количество детей пьют много кофе
* Сотрудники, которые уволятся в следующем году, часто имеют загранпаспорт
* Сотрудники с большим стажем работы, которые уволятся в следующем году, живут далеко от работы
* Высокие сотрудники, которые уволятся в следующем году, часто перерабатывают
* Уход сотрудника в следующем году увеличивает его шансы на переработки и увеличивает стоимость ланча в кафе


## Задание 2. Дайте интерпретацию полученных результатов.

В ходе анализа ассоциативных правил было выделено несколько правил, которые можно разделить на очевидные, логичные, но не очевидные, и нелогичные правила. С помощью логичных, но не очевидных правил мы можем понять, например, как удержать сотрудника от ухода с работы. Благодаря найденным правилам мы знаем, что для снижения шанса ухода сотрудника необходимо, прежде всего, снизить обьем переработок, а также повысить зарплату до приемлемого уровня.

Также необходимо отметить, что качество ассоциативных правил удалось увеличить за счет удаления высоко коррелируемых переменных из датасета.